# Import dependencies

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from typing import List
import os
from dotenv import load_dotenv

/Users/sumanyadav/Desktop/PROJECT/Lernix/backend/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise EnvironmentError("OPENAI_API_KEY environment variable not set")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# 1. Chunk text
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        length_function=len
    )
    return splitter.split_text(text)

In [4]:
# 2. Convert chunks to Documents
def convert_to_document(chunks: List[str]) -> List[Document]:
    return [Document(page_content=chunk) for chunk in chunks]

In [5]:
# 3. Create Hybrid Retriever
def create_retriever(docs: List[Document]):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    dense_vectorstore = FAISS.from_documents(docs, embeddings)
    dense_retriever = dense_vectorstore.as_retriever(search_kwargs={"k": 10})

    sparse_retriever = BM25Retriever.from_documents(docs)
    sparse_retriever.k = 3

    return EnsembleRetriever(
        retrievers=[dense_retriever, sparse_retriever],
        weights=[0.7, 0.3]
    )

In [6]:
def ask_question_rag_chain(retriever):
    prompt_text = """
    Answer the question based on the following context:
    
    Context:{context}
    
    Question:{input}
    """

    prompt = PromptTemplate.from_template(prompt_text)
    
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7
    )

    document_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=prompt
    )

    return create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=document_chain
    )

In [7]:
def ask_question(text: str) -> str:
    question = input("Enter your question: ")
    chunks = chunk_text(text)
    docs = convert_to_document(chunks)
    retriever = create_retriever(docs)
    rag_chain = ask_question_rag_chain(retriever)

    response = rag_chain.invoke({"input": question})
    return response["answer"]


In [8]:
text = """
Interpersonal Communication with Strangers
Interpersonal communication with strangers refers to the exchange of information, thoughts, and
feelings between people who do not know each other well. This type of communication commonly
occurs in daily life, such as when speaking with classmates, coworkers, shopkeepers, service
providers, or people met for the first time. Effective communication with strangers requires clarity,
politeness, and respect. Since there is little or no prior relationship, first impressions are very
important. Simple actions like maintaining eye contact, using a friendly tone, and practicing good
listening skills help create a positive interaction. Clear language and appropriate body posture also
reduce misunderstandings. Active listening plays a key role in interpersonal communication. It
involves paying attention, avoiding interruptions, and responding thoughtfully. When communicating
with strangers, asking relevant questions and acknowledging their responses shows interest and
respect, helping build trust quickly. Nonverbal communication is equally important. Facial
expressions, gestures, and personal space can strongly influence how a message is received.
Being aware of cultural differences is essential, as gestures and communication styles may vary
across cultures. Safety and boundaries should always be maintained while interacting with
strangers. Personal information should not be shared unnecessarily, and communication should
remain appropriate and professional. Overall, effective interpersonal communication with strangers
helps develop social skills, build confidence, and create positive connections in everyday life.
"""

answer = ask_question(text=text)
print(answer)


Interpersonal communication refers to the exchange of information, thoughts, and feelings between individuals who do not know each other well. It involves interactions that commonly occur in daily life, such as conversations with classmates, coworkers, shopkeepers, service providers, or people met for the first time. Effective interpersonal communication emphasizes clarity, politeness, respect, active listening, and awareness of nonverbal cues and cultural differences to foster positive connections and build trust.
